In [ ]:
import requests

In [ ]:
url = "https://cluster.elasticsearch.dataesr.ovh/bsso-publications/_search"

In [ ]:
header= {"authorization": open("../../.env_es_bsso", "r").read().strip() }
AGG_SIZE = 15

### Sommaire

* [Publications](#publications)
    * [Général](#publi_general)
        * [Dynamique d'ouverture](#dynamique_ouverture)
        * [Voies d'ouverture](#voie_ouverture)
        * [Genre](#genre)
        * [Langue](#langue)
        * [Financement](#financement)
        * [COI](#coi)
    * [Discipline](#publi_disciplines)
        * [Dynamique d'ouverture par discipline](#dynamique_ouverture_disciplines)
        * [Voies d'ouverture par discipline](#voie_ouverture_disciplines)
    * [Editeurs](#publi_editeurs)
        * [Dynamique d'ouverture pour editeur](#dynamique_ouverture_editeurs)
        * [Voies d'ouverture par discipline](#voie_ouverture_editeurs)
        * [Politique d'ouverture par discipline](#politique_ouverture_editeurs)
        * [Poids des revues prédatrices](#predateur)
        * [Répartition par licence ouverte](#licence)
        * [APC](#apc)
     


# 1. Publications <a class="anchor" id="publications"></a>

## 1.1 Général <a class="anchor" id="publi_general"></a>

### 1.1.1 Dynamique d'ouverture (2 graphes, barres et lignes) <a class="anchor" id="dynamique_ouverture"></a>

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.1.2 Voies d'ouverture <a class="anchor" id="voie_ouverture"></a>

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_oa_host_type": {
          "terms": {
            "field": "oa_details." + year + ".oa_host_type.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.1.3 Genre de publications <a class="anchor" id="genre"></a>

In [ ]:
year = "2021Q1" # latest
params = {
  "aggs": {
    "by_publication_genre": {
      "terms": {
        "field": "genre.keyword"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.1.4 Langue <a class="anchor" id="langue"></a>

In [ ]:
year = "2021Q1" #latest
params = {
  "aggs": {
    "by_publication_genre": {
      "terms": {
        "field": "language.keyword"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.1.6 Déclaration de financement <a class="anchor" id="financement"></a>

barres

In [ ]:
year = "2021Q1" #latest
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          },
          "aggs": {
              "by_has_grant": {
                  "terms": {
                      "field": "has_grant"
                  }
              }
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

treemap (par agence)

In [ ]:
year = "2021Q1" #latest
params = {
  "aggs": {
    "by_grant_agency": {
      "terms": {
        "field": "grants.agency.keyword"
      },
      "aggs": {
        "by_oa_host_type": {
          "terms": {
            "field": "oa_details." + year + ".oa_host_type.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.1.7 COI <a class="anchor" id="coi"></a>

In [ ]:
year = "2021Q1" #latest
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_has_coi": {
          "terms": {
            "field": "has_coi"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

## 1.2 Disciplines <a class="anchor" id="publi_disciplines"></a>

### 1.2.1 Dynamique par discipline  <a class="anchor" id="dynamique_ouverture_disciplines"></a>

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_discipline": {
      "terms": {
        "field": "bsso_fields.keyword"
      },
      "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "publication_year"
          },
          "aggs": {
            "by_is_oa": {
              "terms": {
                "field": "oa_details." + year + ".is_oa"
              }
            }
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.2.2 Voie d'ouverture par discipline  <a class="anchor" id="voie_ouverture_disciplines"></a>

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_oa_host_type": {
          "terms": {
            "field": "oa_details." + year + ".oa_host_type.keyword"
          },
          "aggs": {
            "by_discipline": {
              "terms": {
                "field": "bsso_fields.keyword"
              }
            }
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

## 1.3 Editeurs <a class="anchor" id="publi_editeurs"></a>

### 1.3.1 Dynamique pour editeurs  <a class="anchor" id="dynamique_ouverture_editeurs"></a>

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
        { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       # ,{ "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.3.2 Voie d'ouverture pour editeurs <a class="anchor" id="voie_ouverture_editeurs"></a>


In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
        { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       # ,{ "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_oa_colors": {
          "terms": {
            "field": "oa_details." + year + ".oa_colors.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.3.3 Politique d'ouverture par editeur <a class="anchor" id="politique_ouverture_editeurs"></a>

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
     #   { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       { "term": { "publication_year": 2020   }} ## pour une publication year donnée
      ]
    }
  },
  "aggs": {
    "by_publisher": {
      "terms": {
        "field": "publisher.keyword"
      },
      "aggs": {
        "by_oa_color": {
          "terms": {
            "field": "oa_details." + year + ".oa_colors.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.3.4 Poids des revues prédatrices <a class="anchor" id="predateur"></a>

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_predatory": {
          "terms": {
            "field": "predatory_journal"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.3.5 Licences <a class="anchor" id="licence"></a>


global

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
     #   { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       { "term": { "publication_year": 2020   }} ## pour une publication year donnée
    #  ,{ "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_licence": {
      "terms": {
        "field": "oa_details." + year + ".licence_publisher.keyword"
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

par éditeur

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
     #   { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       { "term": { "publication_year": 2020   }} ## pour une publication year donnée
      ]
    }
  },
  "aggs": {
    "by_publisher": {
      "terms": {
        "field": "publisher.keyword"
      },
      "aggs": {
        "by_licence": {
          "terms": {
            "field": "oa_details." + year + ".licence_publisher.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

### 1.3.6 APC <a class="anchor" id="apc"></a>


somme des apc par an

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {

  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "apc": {
          "sum": {
            "field": "amount_apc_EUR"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

distribution des tarifs

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
    
"query": {
    "bool": {
      "filter": [
  #  { "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },

  "aggs": {
        "tarif": {
          "histogram": {
            "field": "amount_apc_EUR",
            "interval": 250
          }
        }
      }
    }


requests.get(url, json=params, headers=header).json()['aggregations']

violin : 1. one request for density

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
    
"query": {
    "bool": {
      "filter": [
  #  { "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
            "tarif": {
              "histogram": {
                "field": "amount_apc_EUR",
                "interval": 250
              }
            }
      }
      }
    }
}


requests.get(url, json=params, headers=header).json()['aggregations']

violin : second request for percentiles

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
    
"query": {
    "bool": {
      "filter": [
  #  { "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
            "tarif_percentiles": {
              "percentiles": {
                "field": "amount_apc_EUR"
              }
            }
      }
      }
    }
}


requests.get(url, json=params, headers=header).json()['aggregations']

## 1.4 Les archives

### 1.4.1 Quelle est la dynamique d’ouverture de la santé parmi les archives ?

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "term": {
          "oa_details." + year + ".oa_host_type": "repository"
        }
      }]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']['by_publication_year']['buckets']

_Question : pourquoi `oa_details` n'est pas un simple tableau d'objects avec un champ "millésime" (ou autre) afin de pouvoir faire une aggrégation sur ce champ et donc faire une grosse requête plutôt qu'une requête par millésime ?_ 

In [ ]:
### 1.4.2 Quelles archives ouvertes sont les plus utilisées en santé ?

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "term": {
          "oa_details." + year + ".oa_host_type": "repository"
        }
      }]
    }
  },
  "aggs": {
    "by_repository": {
      "terms": {
        "field": "oa_details." + year + ".repositories.keyword",
        "missing": "MISSING",
        "size": AGG_SIZE
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']['by_repository']['buckets']

In [ ]:
_Erreur : Pour certaines publications, il manque le champ "oa_details.year.repositories.keyword". ex: https://cluster.elasticsearch.dataesr.ovh/bsso-publications/_search?q=_id:1573. `{'key': 'MISSING', 'doc_count': 257}`_
=> Ca devrait être réparé prochainement

In [ ]:
### 1.4.3 Quelle est la dynamique de dépôt par archive ouverte en santé ?

In [ ]:
year = "2021Q1"
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "term": {
          "oa_details." + year + ".oa_host_type": "repository"
        }
      }]
    }
  },
  "aggs": {
    "by_discipline": {
      "terms": {
        "field": "oa_details." + year + ".repositories.keyword",
        "missing": "MISSING",
        "size": AGG_SIZE
      },
      "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "publication_year"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']['by_discipline']['buckets']

In [ ]:
_Question : Cette requête est basée sur le millésime "2021Q1". Est-ce bien la façon dont ce graphe a été créé ?_

In [ ]:
### 1.4.4 Quelle place occupe HAL dans la dynamique des archives ouvertes en santé ?

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "term": {
          "oa_details." + year + ".oa_host_type": "repository"
        }
      }]
    }
  },
  "aggs": {
    "by_discipline": {
      "terms": {
        "field": "oa_details." + year + ".repositories.keyword",
        "missing": "MISSING",
        "size": AGG_SIZE
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']['by_discipline']['buckets']

In [ ]:
_Idée : Should post process to group / sum all repositories other than 'HAL'_

In [ ]:
## 1.5 Les affiliations

In [ ]:
### 1.5.1 Taux d’ouverture des publications françaises, dans le domaine de la santé, par millésime tous types d’établissements confondus

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
filter = '*' # *, university, hospital, cnrs, inserm
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "wildcard": {
          "french_affiliations_types": filter
        }
      }]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']['by_publication_year']['buckets']

In [ ]:
### 1.5.2 Evolution du taux d’ouverture des publications en santé par types d’établissement entre millésimes (Les données ne sont pas encore prêtes)

In [ ]:
### 1.5.3 Quel impact le pays d’affiliation des auteurs a-t-il sur le taux d’ouverture en santé ?

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "size": 0,
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_author_useful_rank_fr": {
          "terms": {
            "field": "author_useful_rank_fr"
          },
          "aggs": {
            "by_is_oa": {
              "terms": {
                "field": "oa_details." + year + ".is_oa"
              }
            }
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']['by_publication_year']['buckets']

In [ ]:
### 1.5.4 Classement des 10 pays d'affiliation des auteurs de rang utile avec lesquels la France collabore le plus, selon le taux d’accès ouvert de leurs publications en santé 

In [ ]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "term": {
          "author_useful_rank_fr": "true"
        }
      }, {
        "term": {
          "oa_details." + year + ".is_oa": "true"
        }
      }]
    }
  },
  "aggs": {
    "by_country": {
      "terms": {
        "field": "affiliations.countries.keyword",
        "exclude": "FR",
        "size": AGG_SIZE,
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']['by_country']['buckets']